In [1]:
import numpy as np
import pandas as pd
import primer3 as pm

Creating DataFrame

In [2]:
with open("inflammation_consensus.fasta.txt", "r") as f:
    lines = f.readlines()

clonotypes = [line.strip() for i, line in enumerate(lines) if i % 2 == 0]
seqs = [line.strip() for i, line in enumerate(lines) if i % 2 == 1]

df = pd.DataFrame({"CC": clonotypes, "Seq": seqs})
df.head()

,CC,Seq
0,>clonotype1_consensus_1,GGGAGACCTTGCCTATGGGACCATAGGATGCTAGCATACCCCTCCT...
1,>clonotype1_consensus_2,TGACACAACAGCCCAGGGACTGGTTACTTGCTTCTGTCTCCAGCAT...
2,>clonotype2_consensus_1,TCTCGTGCTTACGTGGAGTTTCTATGAGTGAAGCCACTGCCTCATC...
3,>clonotype2_consensus_2,GGGTTAGGAATGCAACCACAAGGTGTCAGGAGATCTTAACTGCTGG...
4,>clonotype2_consensus_3,CTCCTTTTGCTGGCTTGAAGTGTGAATCTTCAGTGAAAAGAGTAAT...


In [3]:
'''
alphas = []
betas = []
for seq in seqs:
    if 'ACATCCAGAACCCAGAACCT' in seq:
        alphas.append(seq)
    else:
        betas.append(seq)
'''
t = df['Seq'].str.contains('ACATCCAGAACCCAGAACC')
alpha = df[t]
beta = df[~t]

alpha_orig = alpha.copy(deep = True)
beta_orig = beta.copy(deep = True)

In [4]:
rev = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}
def rev_comp(seq):
    new_seq = "".join(map(rev.get, seq))  # Convert to string
    return new_seq[::-1]
rev_comp('ATG')

'CAT'

In [5]:
def no_repeat_neighbors(seq, index, direction):
    """
    Returns True if the nucleotide at 'index' is NOT part of a contiguous block
    of three or more of the same nucleotide. Returns False if it IS part of such a block.
    """
    nucleotide = seq[index]
    if nucleotide not in ('C', 'G'):
        return True
    
    if direction == 'left':
        if nucleotide == seq[index - 1] and seq[index - 1] == seq[index - 2] and seq[index - 2] == seq[index - 3]:
            return False
        
    if direction == 'right':
        if nucleotide == seq[index + 1] and seq[index + 1] == seq[index + 2] and seq[index + 2] == seq[index + 3]:
            return False
    
    return True

In [6]:
data = {
    'Amino Acid': ['F', 'F', 'L', 'L', 'L', 'L', 'L', 'L', 'I', 'I', 'I', 'M', 'V', 'V', 'V', 'V', 'S', 'S', 'S', 'S', 'S', 'S', 'P', 'P', 'P', 'P', 'T', 'T', 'T', 'T', 'A', 'A', 'A', 'A', 'Y', 'Y', 'H', 'H', 'Q', 'Q', 'N', 'N', 'K', 'K', 'D', 'D', 'E', 'E', 'C', 'C', 'W', 'R', 'R', 'R', 'R', 'R', 'R', 'G', 'G', 'G', 'G', '*', '*', '*'],
    'Triplet': ['TTC', 'TTT', 'CTG', 'CTC', 'CTA', 'TTG', 'TTA', 'CTT', 'ATC', 'ATT', 'ATA', 'ATG', 'GTG', 'GTC', 'GTT', 'GTA', 'AGC', 'AGT', 'TCC', 'TCT', 'TCA', 'TCG', 'CCC', 'CCT', 'CCA', 'CCG', 'ACC', 'ACT', 'ACA', 'ACG', 'GCC', 'GCT', 'GCA', 'GCG', 'TAC', 'TAT', 'CAC', 'CAT', 'CAG', 'CAA', 'AAC', 'AAT', 'AAG', 'AAA', 'GAC', 'GAT', 'GAG', 'GAA', 'TGC', 'TGT', 'TGG', 'AGG', 'AGA', 'CGG', 'CGA', 'CGC', 'CGT', 'GGC', 'GGT', 'GGG', 'GGA', 'TGA', 'TAA', 'TAG'],
    'Frequency': [0.57, 0.43, 0.39, 0.20, 0.08, 0.13, 0.06, 0.13, 0.50, 0.34, 0.22, 1.00, 0.46, 0.25, 0.17, 0.12, 0.24, 0.15, 0.22, 0.19, 0.14, 0.05, 0.31, 0.30, 0.28, 0.10, 0.35, 0.25, 0.29, 0.11, 0.38, 0.29, 0.26, 0.10, 0.58, 0.43, 0.60, 0.40, 0.75, 0.25, 0.57, 0.43, 0.61, 0.39, 0.56, 0.44, 0.60, 0.26, 0.52, 0.48, 1.00, 0.22, 0.21, 0.19, 0.18, 0.12, 0.09, 0.33, 0.18, 0.26, 0.23, 0.52, 0.26, 0.22]
}

aas = pd.DataFrame(data)
aas = aas.sort_values(by=['Amino Acid', 'Frequency'], ascending=[True, False])

codon_dict = aas.groupby('Amino Acid')['Triplet'].apply(list).to_dict()
print(codon_dict)

{'*': ['TGA', 'TAA', 'TAG'], 'A': ['GCC', 'GCT', 'GCA', 'GCG'], 'C': ['TGC', 'TGT'], 'D': ['GAC', 'GAT'], 'E': ['GAG', 'GAA'], 'F': ['TTC', 'TTT'], 'G': ['GGC', 'GGG', 'GGA', 'GGT'], 'H': ['CAC', 'CAT'], 'I': ['ATC', 'ATT', 'ATA'], 'K': ['AAG', 'AAA'], 'L': ['CTG', 'CTC', 'TTG', 'CTT', 'CTA', 'TTA'], 'M': ['ATG'], 'N': ['AAC', 'AAT'], 'P': ['CCC', 'CCT', 'CCA', 'CCG'], 'Q': ['CAG', 'CAA'], 'R': ['AGG', 'AGA', 'CGG', 'CGA', 'CGC', 'CGT'], 'S': ['AGC', 'TCC', 'TCT', 'AGT', 'TCA', 'TCG'], 'T': ['ACC', 'ACA', 'ACT', 'ACG'], 'V': ['GTG', 'GTC', 'GTT', 'GTA'], 'W': ['TGG'], 'Y': ['TAC', 'TAT']}


In [7]:
#Frequency doesn't matter, this is just one change. We just need to make the change and keep whatever nucleotide is there, the same
def preventer(seq, index, direction):
    if (no_repeat_neighbors(seq, index, direction)):
        return True, seq
    seq2 = ''
    nucleotide = seq[index]
    point = index % 3
    if direction == 'left':
        right = 2 - point + index
        codons = [seq[right - 5:right - 2], seq[right - 2:right + 1]]
        aa = [aas[aas['Triplet'] == codons[0]]['Amino Acid'].values[0], aas[aas['Triplet'] == codons[1]]['Amino Acid'].values[0]]
        choices_1 = list(aas[aas['Amino Acid'] == aa[0]]['Triplet'])
        choices_2 = list(aas[aas['Amino Acid'] == aa[1]]['Triplet'])
        for choice in choices_2:
            seq2 = seq[:right - 5] + choice + seq[right - 2:]
            if no_repeat_neighbors(seq2, index, 'left'):
                return True, seq2
        for choice in choices_1:
            seq2 = seq[:right - 2] + choice + seq[right+1:]
            if seq2[index] != nucleotide:
                continue
            if no_repeat_neighbors(seq2, index, 'left'):
                return True, seq2
        
    if direction == 'right':
        right = 2 - point + index
        codons = [seq[right + 1:right + 4], seq[right - 2:right + 1]]
        aa = [aas[aas['Triplet'] == codons[0]]['Amino Acid'].values[0], aas[aas['Triplet'] == codons[1]]['Amino Acid'].values[0]]
        choices_1 = list(aas[aas['Amino Acid'] == aa[0]]['Triplet'])
        choices_2 = list(aas[aas['Amino Acid'] == aa[1]]['Triplet'])
        for choice in choices_1:
            seq2 = seq[:right+1] + choice + seq[right+4:]
            if no_repeat_neighbors(seq2, index, 'right'):
                return True, seq2
        for choice in choices_2:
            seq2 = seq[:right - 2] + choice + seq[right+1:]
            if seq2[index] != nucleotide:
                continue
            if no_repeat_neighbors(seq2, index, 'right'):
                return True, seq2
    return False, None
            


t = preventer('TGAGTCCCCGATGCCCCTGC', 7, 'left')

In [8]:
with open("Missing_Alphas.txt", "w") as f:
    f.write("Unable to find working sequences for clonotypes: \n")

with open("Missing_Betas.txt", "w") as f:
    f.write("Unable to find working sequences for clonotypes: \n")

def missing(tp, clono, reason):
    with open(f"Missing_{tp}.txt", "a") as f:
        f.write(f"{clono}: {reason}\n")

TRAV

In [9]:
with open("TRAVs.fasta.txt", "r") as f:
    lines = f.readlines()

name = [line.strip() for i, line in enumerate(lines) if i % 2 == 0]
travs = [line.strip() for i, line in enumerate(lines) if i % 2 == 1]

def TRAV(seq, CC):
    end_ind = None
    trav_poss = []
    for trav in travs:
        if trav[:30] in seq: #Going to assume we need up to 100 nucleotides of trav to find the unique one it corresponds to (CHECK THIS)
            trav_poss.append(trav)
            
    if len(trav_poss) == 0:
        missing('Alphas', CC, 'Could not find TRAV that worked')
        return None, None, None

    start_ind = seq.find(trav_poss[0][:30])
    temp_inds = [len(trav) for trav in trav_poss]
    while temp_inds[0] > 200:
        for ind in range(len(trav_poss)):
            if seq[start_ind:start_ind + temp_inds[ind]] != trav_poss[ind][:temp_inds[ind]]:
                temp_inds[ind]-=1
            else:
                end_ind = start_ind + temp_inds[ind]
                return start_ind, end_ind, seq[start_ind:end_ind]
    if end_ind == None:
        missing('Alphas', CC, 'Could not find TRAV Primer that worked')
        return None, None, None
    return start_ind, end_ind, seq[start_ind:end_ind]

alpha = alpha.copy()
#alpha[['TRAV_start_ind', 'TRAV_end_ind', 'TRAV']] = alpha['Seq'].apply(lambda seq: pd.Series(TRAV(seq)))
alpha[['TRAV_start_ind', 'TRAV_end_ind', 'TRAV']] = alpha.apply(lambda row: pd.Series(TRAV(row['Seq'], row['CC'])), axis = 1)
#alpha = alpha.dropna()
alpha


,CC,Seq,TRAV_start_ind,TRAV_end_ind,TRAV
1,>clonotype1_consensus_2,TGACACAACAGCCCAGGGACTGGTTACTTGCTTCTGTCTCCAGCAT...,59.0,387.0,ATGAAGAGGCTGCTGTGTTCTCTGCTGGGGCTTCTGTGCACCCAGG...
3,>clonotype2_consensus_2,GGGTTAGGAATGCAACCACAAGGTGTCAGGAGATCTTAACTGCTGG...,238.0,580.0,ATGAAAACATATGCTCCTACATTATTCATGTTTCTATGGCTGCAGC...
4,>clonotype2_consensus_3,CTCCTTTTGCTGGCTTGAAGTGTGAATCTTCAGTGAAAAGAGTAAT...,44.0,380.0,ATGCATTCCTTACATGTTTCACTAGTGTTCCTCTGGCTTCAACTAG...
6,>clonotype3_consensus_2,GAACTGCATTTCCTGCTGGGGTGGTTACACCACACAGACCAGTTAC...,135.0,483.0,ATGCTGATTCTAAGCCTGTTGGGAGCAGCCTTTGGCTCCATTTGTT...
8,>clonotype4_consensus_2,TGGGGGAGGTCAGGGAGAGTCTCTTTTCTAGTTACCAACTTAGAGT...,85.0,417.0,ATGAAATCCTTTAGTATTTCCCTAGTGGTCCTGTGGCTTCAGCTAA...
...,...,...,...,...,...
21129,>clonotype9965_consensus_2,AGAGAGCTGAGGTGCCTGTGAAGTTGGAGTCAGTGTTCTGTGGGAG...,89.0,447.0,ATGGACAAGATCCTGACAGCATCGTTTTTACTTCTAGGCCTTCACC...
21130,>clonotype9966_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...
21131,>clonotype9967_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...
21132,>clonotype9968_consensus_1,GGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATG...,4.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...


CDR3a Reverse Primer

In [10]:
def CDR_Reverse(seq, CC):
    ind = seq.find('ACATCCAGAACCCAGAACCT')
    rev_prime_ending = ind + 20
    ind = ind - 40
    point = ind + min(seq[ind:].find('C'), seq[ind:].find('G'))
    end_point = None
    rev_primer = None
    for end in range(point + 17, point + 31):
        if (seq[end - 1] == 'C' or seq[end - 1] == 'G') and (pm.calc_tm(seq[point:end]) >= 54 and pm.calc_tm(seq[point:end]) <= 58):
            found, temp_seq = preventer(seq, end - 1, 'left')
            if found:
                seq = temp_seq
            else:
                continue
            end_point = end
            rev_primer = seq[point:rev_prime_ending]
            break
    if end_point == None or rev_primer == None:
        missing('Alphas', CC, 'Could not find CDR Reverse Primer that worked')
        return None, None, None, None
    return point, end_point, seq[point:end_point], pm.calc_tm(seq[point:end_point]), rev_comp(rev_primer), 

alpha = alpha.copy()
#alpha[['CDR3a_overlap_start_ind', 'CDR3a_overlap_end_ind', 'CDR3a_overlap_seq', 'CDR3a_overlap_Tm', 'CDR3a_Rev_Primer']] = alpha['Seq'].apply(lambda seq: pd.Series(CDR_Reverse(seq)))
alpha[['CDR3a_overlap_start_ind', 'CDR3a_overlap_end_ind', 'CDR3a_overlap_seq', 'CDR3a_overlap_Tm', 'CDR3a_Rev_Primer']] = alpha.apply(lambda row: pd.Series(CDR_Reverse(row['Seq'], row['CC'])), axis = 1)
#alpha = alpha.dropna()
alpha  

,CC,Seq,TRAV_start_ind,TRAV_end_ind,TRAV,CDR3a_overlap_start_ind,CDR3a_overlap_end_ind,CDR3a_overlap_seq,CDR3a_overlap_Tm,CDR3a_Rev_Primer
1,>clonotype1_consensus_2,TGACACAACAGCCCAGGGACTGGTTACTTGCTTCTGTCTCCAGCAT...,59.0,387.0,ATGAAGAGGCTGCTGTGTTCTCTGCTGGGGCTTCTGTGCACCCAGG...,410.0,430.0,CTCATCTTTGGACTGGGGAC,55.485372,AGGTTCTGGGTTCTGGATGTCTGGTTGCACTTGTAAAGTTGTCCCC...
3,>clonotype2_consensus_2,GGGTTAGGAATGCAACCACAAGGTGTCAGGAGATCTTAACTGCTGG...,238.0,580.0,ATGAAAACATATGCTCCTACATTATTCATGTTTCTATGGCTGCAGC...,605.0,622.0,CCTTTGGGGATGGGACC,54.257077,AGGTTCTGGGTTCTGGATGTTTGGCTTCACTGTGAGCACGGTCCCA...
4,>clonotype2_consensus_3,CTCCTTTTGCTGGCTTGAAGTGTGAATCTTCAGTGAAAAGAGTAAT...,44.0,380.0,ATGCATTCCTTACATGTTTCACTAGTGTTCCTCTGGCTTCAACTAG...,404.0,424.0,CTCATCTTTGGACTGGGGAC,55.485372,AGGTTCTGGGTTCTGGATGTCTGGTTGCACTTGTAAAGTTGTCCCC...
6,>clonotype3_consensus_2,GAACTGCATTTCCTGCTGGGGTGGTTACACCACACAGACCAGTTAC...,135.0,483.0,ATGCTGATTCTAAGCCTGTTGGGAGCAGCCTTTGGCTCCATTTGTT...,503.0,523.0,CTTCTTTGGTGATGGGACGC,57.090331,AGGTTCTGGGTTCTGGATGTTGGGCTTCACCACCAGCTGCGTCCCA...
8,>clonotype4_consensus_2,TGGGGGAGGTCAGGGAGAGTCTCTTTTCTAGTTACCAACTTAGAGT...,85.0,417.0,ATGAAATCCTTTAGTATTTCCCTAGTGGTCCTGTGGCTTCAGCTAA...,439.0,459.0,CTTTACTTCGGATCTGGCAC,54.117413,AGGTTCTGGGTTCTGGATGTTTGGCTCTACAGTGAGTTTGGTGCCA...
...,...,...,...,...,...,...,...,...,...,...
21129,>clonotype9965_consensus_2,AGAGAGCTGAGGTGCCTGTGAAGTTGGAGTCAGTGTTCTGTGGGAG...,89.0,447.0,ATGGACAAGATCCTGACAGCATCGTTTTTACTTCTAGGCCTTCACC...,470.0,492.0,CTGACATTTGGGAAAGGAACTC,55.087404,AGGTTCTGGGTTCTGGATGTTTGGAGTCACAGTTAAGAGAGTTCCT...
21130,>clonotype9966_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,408.0,426.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...
21131,>clonotype9967_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,408.0,426.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...
21132,>clonotype9968_consensus_1,GGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATG...,4.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,406.0,424.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...


CDR3a forward and forward outer

In [11]:
def CDRA_Forwards(seq, trav_overlap_end, cdr_overlap_end, CC):
    if pd.isna(trav_overlap_end) or pd.isna(cdr_overlap_end):
        return None, None, None, None, None, None
    trav_overlap_end, cdr_overlap_end = int(trav_overlap_end), int(cdr_overlap_end)
    trav_overlap_beg = trav_overlap_end - 16
    while pm.calc_tm(seq[trav_overlap_beg:trav_overlap_end]) < 49:
        trav_overlap_beg-=1
        if trav_overlap_beg < 0:
            missing('Alphas', CC, 'Could not find TRAV Overlap that worked')
            return None, None, None, None, None, None
    cdr_forward_beg = trav_overlap_beg
    while cdr_overlap_end - cdr_forward_beg > 60 or no_repeat_neighbors(seq, cdr_forward_beg, 'right') == False:
        cdr_forward_beg+=1
        if cdr_forward_beg >= cdr_overlap_end:
            missing('Alphas', CC, 'Could not find CDRA Forward Primer that worked')
            return trav_overlap_beg, trav_overlap_end, seq[trav_overlap_beg:trav_overlap_end], None, None, None
    cdr_forward = seq[cdr_forward_beg:cdr_overlap_end]
    cdr_forward_outer_end_ind = trav_overlap_beg + 16
    #while pm.calc_tm(seq[trav_overlap_beg:cdr_forward_outer_end_ind]) < 49 or (pm.calc_tm(seq[cdr_forward_beg:cdr_forward_outer_end_ind]) < 54 or pm.calc_tm(seq[cdr_forward_beg:cdr_forward_outer_end_ind]) > 58) or (seq[cdr_forward_outer_end_ind - 1] not in ('C','G')): #or no_repeat_neighbors(seq, cdr_forward_outer_end_ind - 1, 'left') == False:
    while True:
        if pm.calc_tm(seq[trav_overlap_beg:cdr_forward_outer_end_ind]) >= 49 and (pm.calc_tm(seq[cdr_forward_beg:cdr_forward_outer_end_ind]) >= 54 and pm.calc_tm(seq[cdr_forward_beg:cdr_forward_outer_end_ind]) <= 58) and (seq[cdr_forward_outer_end_ind - 1] in ('C','G')):
            found, temp_seq = preventer(seq, cdr_forward_outer_end_ind - 1, 'left')
            if found:
                seq = temp_seq
                break
        cdr_forward_outer_end_ind+=1
        if cdr_forward_outer_end_ind > cdr_overlap_end:
            missing('Alphas', CC, 'Could not find CDRA Forward Outer Primer that worked')
            return trav_overlap_beg, trav_overlap_end, seq[trav_overlap_beg:trav_overlap_end], cdr_forward, None, None
    cdr_forward_outer = seq[trav_overlap_beg:cdr_forward_outer_end_ind]
    
    temp_cdr = cdr_forward_outer
    while temp_cdr not in cdr_forward:
        temp_cdr = temp_cdr[1:]

    return trav_overlap_beg, trav_overlap_end, seq[trav_overlap_beg:trav_overlap_end], cdr_forward, cdr_forward_outer, pm.calc_tm(temp_cdr)
    
alpha = alpha.copy()
alpha[['TRAV_overlap_start_ind', 'TRAV_overlap_end_ind', 'TRAV_overlap_seq', 'CDR3a_forward', 'CDR3a_forward_outer', 'CDR3a_forward_outer_Tm']] = alpha.apply(
    lambda row: pd.Series(CDRA_Forwards(row['Seq'], row['TRAV_end_ind'], row['CDR3a_overlap_end_ind'], row['CC'])),
    axis=1
)
#alpha = alpha.dropna()
alpha  

,CC,Seq,TRAV_start_ind,TRAV_end_ind,TRAV,CDR3a_overlap_start_ind,CDR3a_overlap_end_ind,CDR3a_overlap_seq,CDR3a_overlap_Tm,CDR3a_Rev_Primer,TRAV_overlap_start_ind,TRAV_overlap_end_ind,TRAV_overlap_seq,CDR3a_forward,CDR3a_forward_outer,CDR3a_forward_outer_Tm
1,>clonotype1_consensus_2,TGACACAACAGCCCAGGGACTGGTTACTTGCTTCTGTCTCCAGCAT...,59.0,387.0,ATGAAGAGGCTGCTGTGTTCTCTGCTGGGGCTTCTGTGCACCCAGG...,410.0,430.0,CTCATCTTTGGACTGGGGAC,55.485372,AGGTTCTGGGTTCTGGATGTCTGGTTGCACTTGTAAAGTTGTCCCC...,366.0,387.0,GCACTTATTTCTGTGCTATAG,TTATTTCTGTGCTATAGGGGGAACAGGCAATACCGGAAAACTCATC...,GCACTTATTTCTGTGCTATAGGGGGAAC,55.909889
3,>clonotype2_consensus_2,GGGTTAGGAATGCAACCACAAGGTGTCAGGAGATCTTAACTGCTGG...,238.0,580.0,ATGAAAACATATGCTCCTACATTATTCATGTTTCTATGGCTGCAGC...,605.0,622.0,CCTTTGGGGATGGGACC,54.257077,AGGTTCTGGGTTCTGGATGTTTGGCTTCACTGTGAGCACGGTCCCA...,564.0,580.0,CTTCTGTGCTGCAAGT,CTTCTGTGCTGCAAGTCCGAGCACCAATACAGGCAAATTAACCTTT...,CTTCTGTGCTGCAAGTCCG,57.040949
4,>clonotype2_consensus_3,CTCCTTTTGCTGGCTTGAAGTGTGAATCTTCAGTGAAAAGAGTAAT...,44.0,380.0,ATGCATTCCTTACATGTTTCACTAGTGTTCCTCTGGCTTCAACTAG...,404.0,424.0,CTCATCTTTGGACTGGGGAC,55.485372,AGGTTCTGGGTTCTGGATGTCTGGTTGCACTTGTAAAGTTGTCCCC...,364.0,380.0,CCTCTGTGCAGTGAGC,CCTCTGTGCAGTGAGCCGTATAACAGGCAATACCGGAAAACTCATC...,CCTCTGTGCAGTGAGCC,55.143497
6,>clonotype3_consensus_2,GAACTGCATTTCCTGCTGGGGTGGTTACACCACACAGACCAGTTAC...,135.0,483.0,ATGCTGATTCTAAGCCTGTTGGGAGCAGCCTTTGGCTCCATTTGTT...,503.0,523.0,CTTCTTTGGTGATGGGACGC,57.090331,AGGTTCTGGGTTCTGGATGTTGGGCTTCACCACCAGCTGCGTCCCA...,461.0,483.0,AGTATATTTCTGTGCTATGAGA,TATATTTCTGTGCTATGAGATGGAATAGCAATAACAGAATCTTCTT...,AGTATATTTCTGTGCTATGAGATGGAATAG,54.839399
8,>clonotype4_consensus_2,TGGGGGAGGTCAGGGAGAGTCTCTTTTCTAGTTACCAACTTAGAGT...,85.0,417.0,ATGAAATCCTTTAGTATTTCCCTAGTGGTCCTGTGGCTTCAGCTAA...,439.0,459.0,CTTTACTTCGGATCTGGCAC,54.117413,AGGTTCTGGGTTCTGGATGTTTGGCTCTACAGTGAGTTTGGTGCCA...,401.0,417.0,TCTACCTCTGTGCAGC,TCTACCTCTGTGCAGCCCAGGGGTCTAATTACAACGTGCTTTACTT...,TCTACCTCTGTGCAGCCC,56.203817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21129,>clonotype9965_consensus_2,AGAGAGCTGAGGTGCCTGTGAAGTTGGAGTCAGTGTTCTGTGGGAG...,89.0,447.0,ATGGACAAGATCCTGACAGCATCGTTTTTACTTCTAGGCCTTCACC...,470.0,492.0,CTGACATTTGGGAAAGGAACTC,55.087404,AGGTTCTGGGTTCTGGATGTTTGGAGTCACAGTTAAGAGAGTTCCT...,430.0,447.0,CTACTTCTGTGCAGCAA,ACTTCTGTGCAGCAAACAGTGGAGGCAGCAATTACAAACTGACATT...,CTACTTCTGTGCAGCAAACAG,54.236044
21130,>clonotype9966_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,408.0,426.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...,363.0,381.0,TACTACTGTGCTCTGAGT,TACTGTGCTCTGAGTACTAACACTGGAGCTAACACTGGAAAGCTCA...,TACTACTGTGCTCTGAGTACTAACAC,55.229221
21131,>clonotype9967_consensus_1,GGGGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGA...,6.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,408.0,426.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...,363.0,381.0,TACTACTGTGCTCTGAGT,TACTGTGCTCTGAGTACTAACACTGGAGCTAACACTGGAAAGCTCA...,TACTACTGTGCTCTGAGTACTAACAC,55.229221
21132,>clonotype9968_consensus_1,GGACATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATG...,4.0,381.0,ATGCTAAGAATCATGAACACTGTCGAGATGGGTCTAAAGATGAACT...,406.0,424.0,CTCACGTTTGGACACGGC,56.981512,AGGTTCTGGGTTCTGGATGTTTGGATGGACCCTAAGGATGGTGCCG...,363.0,381.0,CTACTGTGCTCTGAGTGA,TACTGTGCTCTGAGTGATCGAACTGGAGCTAACACTGGAAAGCTCA...,CTACTGTGCTCTGAGTGATCG,54.663627


CDR3b

In [12]:
with open("TRBVs.fasta.txt", "r") as f:
    lines = f.readlines()

name = [line.strip() for i, line in enumerate(lines) if i % 2 == 0]
trbvs = [line.strip() for i, line in enumerate(lines) if i % 2 == 1]
missing_num = []

def CDRB(seq, CC):
    cdrb_start_ind = 0
    for trbv in trbvs:
        trbv_start_ind = seq.find(trbv[:80])
        temp_ind = len(trbv)
        if trbv_start_ind != -1:
            while seq[trbv_start_ind:trbv_start_ind + temp_ind] != trbv[:temp_ind]:
                temp_ind-=1
                if temp_ind < 200:
                    missing('Betas', CC, 'Could not find TRBV Sequence that worked') 
                    return None, None, None, None, None, None, None, None
            cdrb_start_ind = trbv_start_ind + temp_ind
            break
    trbv_overlap_end_ind = cdrb_start_ind
    trbv_overlap_start_ind = cdrb_start_ind
    while pm.calc_tm(seq[trbv_overlap_start_ind:trbv_overlap_end_ind]) < 54 or pm.calc_tm(seq[trbv_overlap_start_ind:trbv_overlap_end_ind]) > 58:
        trbv_overlap_start_ind-=1
        if trbv_overlap_start_ind < trbv_overlap_end_ind - 100:
            missing('Betas', CC, 'Could not find TRBV Overlap Primer that worked')
            return None, None, None, None, None, None, None, None

    cdrb_end_ind = trbv_overlap_start_ind + 40
    #while ((seq[cdrb_end_ind - 1] not in ('C', 'G')) or (cdrb_end_ind - cdrb_start_ind > 60)): #or (not no_repeat_neighbors(seq, cdrb_end_ind - 1, 'left'))):
    while True:
        if ((seq[cdrb_end_ind - 1] in ('C', 'G')) and (cdrb_end_ind - cdrb_start_ind <= 60)):
            found, temp_seq = preventer(seq, cdrb_end_ind - 1, 'left')
            if found:
                seq = temp_seq
                break
        cdrb_end_ind-=1
        if cdrb_end_ind < trbv_overlap_start_ind:
            missing('Betas', CC, 'Could not find CDR3b Primer that worked')
            return trbv_overlap_start_ind, trbv_overlap_end_ind, seq[trbv_overlap_start_ind:trbv_overlap_end_ind], None, None, None, None, None

    prim_overlap_end_ind = cdrb_end_ind
    prim_overlap_start_ind = prim_overlap_end_ind - 16
    #while pm.calc_tm(seq[prim_overlap_start_ind:cdrb_end_ind]) < 54 or pm.calc_tm(seq[prim_overlap_start_ind:cdrb_end_ind]) > 58 or seq[prim_overlap_start_ind] not in ('C', 'G'): #or (not no_repeat_neighbors(seq, prim_overlap_start_ind, 'right')):
    while True:
        if pm.calc_tm(seq[prim_overlap_start_ind:cdrb_end_ind]) >= 54 and pm.calc_tm(seq[prim_overlap_start_ind:cdrb_end_ind]) <= 58 and seq[prim_overlap_start_ind] in ('C', 'G'):
            found, temp_seq = preventer(seq, prim_overlap_start_ind, 'right')
            if found:
                seq = temp_seq
                break
        prim_overlap_start_ind-=1
        if prim_overlap_end_ind - prim_overlap_start_ind > 30:
            missing('Betas', CC, 'Could not find Primer Overlap that worked')
            return trbv_overlap_start_ind, trbv_overlap_end_ind, seq[trbv_overlap_start_ind:trbv_overlap_end_ind], None, None, None, None, None

    return trbv_overlap_start_ind, trbv_overlap_end_ind, seq[trbv_overlap_start_ind:trbv_overlap_end_ind], prim_overlap_start_ind, prim_overlap_end_ind, seq[prim_overlap_start_ind:prim_overlap_end_ind], pm.calc_tm(seq[prim_overlap_start_ind:prim_overlap_end_ind]), seq[trbv_overlap_start_ind:cdrb_end_ind]#seq[cdrb_start_ind:cdrb_end_ind]

beta = beta.copy()
#beta[['TRBV_overlap_start_ind', 'TRBV_overlap_end_ind', 'TRBV_overlap_seq', 'Prim_overlap_start_ind', 'Prim_overlap_end_ind', 'Primer_overlap_seq', 'Primer_overlap_Tm', 'CDR3b_forward']] = beta['Seq'].apply(lambda seq: pd.Series(CDRB(seq)))
beta[['TRBV_overlap_start_ind', 'TRBV_overlap_end_ind', 'TRBV_overlap_seq', 'Prim_overlap_start_ind', 'Prim_overlap_end_ind', 'Primer_overlap_seq', 'Primer_overlap_Tm', 'CDR3b_forward']] = beta.apply(lambda row: pd.Series(CDRB(row['Seq'], row['CC'])), axis = 1)
#beta = beta.dropna()
beta  


,CC,Seq,TRBV_overlap_start_ind,TRBV_overlap_end_ind,TRBV_overlap_seq,Prim_overlap_start_ind,Prim_overlap_end_ind,Primer_overlap_seq,Primer_overlap_Tm,CDR3b_forward
0,>clonotype1_consensus_1,GGGAGACCTTGCCTATGGGACCATAGGATGCTAGCATACCCCTCCT...,436.0,452.0,TGCCAGCAGCTTAGCC,457.0,476.0,GGACTCCGACTACACCTTC,54.489626,TGCCAGCAGCTTAGCCGGGGGGGACTCCGACTACACCTTC
2,>clonotype2_consensus_1,TCTCGTGCTTACGTGGAGTTTCTATGAGTGAAGCCACTGCCTCATC...,374.0,394.0,GTATCTTTGTGCAAGCAGCT,396.0,414.0,GGCGGATGGAACACCTTG,56.326643,GTATCTTTGTGCAAGCAGCTCTGGCGGATGGAACACCTTG
5,>clonotype3_consensus_1,AATACCCGTCTGGAGCCTGATTCCACCATGAGCTGCAGGCTTCTCC...,352.0,368.0,GTGCCAGCAGCCAAGA,366.0,390.0,GATAACAATTCTGGAAATACGCTC,54.076533,GTGCCAGCAGCCAAGATAACAATTCTGGAAATACGCTC
7,>clonotype4_consensus_1,GAAAACTGGCTTGCTCATGTAGAGAAGTGGTGGAGTGTCTTAACTG...,476.0,494.0,TTCTGTGCCAGCAGTGAT,491.0,516.0,GATAATAGGGGAGAAGTCTTCTTTG,54.571719,TTCTGTGCCAGCAGTGATAATAGGGGAGAAGTCTTCTTTG
9,>clonotype5_consensus_1,AATACCCGTCTGGAGCCTGATTCCACCATGAGCTGCAGGCTTCTCC...,350.0,366.0,TTGTGCCAGCAGCCAA,369.0,387.0,GGGCAAGACACCCAGTAC,55.201463,TTGTGCCAGCAGCCAACTGGGGCAAGACACCCAGTAC
...,...,...,...,...,...,...,...,...,...,...
21119,>clonotype9960_consensus_1,TGGGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATG...,400.0,416.0,TGTGCCAGCGGTGATG,419.0,439.0,GCTGGGGGTTTAACCAAGAC,56.641294,TGTGCCAGCGGTGATGCGGGCTGGGGGTTTAACCAAGAC
21121,>clonotype9961_consensus_1,GGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATGTC...,395.0,411.0,TTCTGTGCCAGCGGTG,NaN,NaN,None,NaN,None
21124,>clonotype9963_consensus_1,GAAGCAGGCTGAGCCTAAAGCACAGAAGGGCATAGCCAACACTGTA...,543.0,562.0,TGTACTTCTGTGCCAGCTC,NaN,NaN,None,NaN,None
21126,>clonotype9964_consensus_1,TGGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATGT...,399.0,415.0,TGTGCCAGCGGTGATG,419.0,436.0,GGGACTGGGGGGTCATC,56.161153,TGTGCCAGCGGTGATGCAATGGGACTGGGGGGTCATC


In [13]:
'''
with open("TRBVs.fasta.txt", "r") as f:
    lines = f.readlines()

name = [line.strip() for i, line in enumerate(lines) if i % 2 == 0]
trbvs = [line.strip() for i, line in enumerate(lines) if i % 2 == 1]
'''
def CDRB_reverse(seq, primer_overlap_start, primer_overlap_end, CC):
    if pd.isna(primer_overlap_start) or pd.isna(primer_overlap_end):
        return None, None, None, None, None, None
    primer_overlap_start, primer_overlap_end = int(primer_overlap_start), int(primer_overlap_end)

    trbc_overlap_start_ind = seq.find('AGGATCTGAGAAATGTGACTC', primer_overlap_end)
    if trbc_overlap_start_ind == -1:
        missing('Betas', CC, 'Could not find TRBC')
        return None, None, None, None, None, None
    
    trbc_overlap_end_ind = trbc_overlap_start_ind + len('AGGATCTGAGAAATGTGACTC')
    cdr3b_rev_primer_start = primer_overlap_start
    cdr3b_rev_primer_end = trbc_overlap_end_ind
    while cdr3b_rev_primer_end - cdr3b_rev_primer_start > 60:
        cdr3b_rev_primer_end-=1

    cdr3b_rev__outer_primer_end = trbc_overlap_end_ind
    cdr3b_rev__outer_primer_start = trbc_overlap_start_ind
    while pm.calc_tm(seq[cdr3b_rev__outer_primer_start:cdr3b_rev_primer_end]) < 54 or pm.calc_tm(seq[cdr3b_rev__outer_primer_start:cdr3b_rev_primer_end]) > 58 or (not no_repeat_neighbors(seq, cdr3b_rev__outer_primer_start, 'right')):
        cdr3b_rev__outer_primer_start-=1
        if cdr3b_rev__outer_primer_start < cdr3b_rev_primer_start:
            missing('Betas', CC, 'Could not find CDR3B Reverse Outer Primer that worked')
            return trbc_overlap_start_ind, trbc_overlap_end_ind, seq[trbc_overlap_start_ind:trbc_overlap_end_ind], rev_comp(seq[cdr3b_rev_primer_start:cdr3b_rev_primer_end]), None, None
    
    cdr3b_rev_primer = rev_comp(seq[cdr3b_rev_primer_start:cdr3b_rev_primer_end])
    cdr3b_rev_out_primer = rev_comp(seq[cdr3b_rev__outer_primer_start:cdr3b_rev__outer_primer_end])
    temp_cdr = cdr3b_rev_out_primer
    while temp_cdr not in cdr3b_rev_primer:
        temp_cdr = temp_cdr[1:]
    return trbc_overlap_start_ind, trbc_overlap_end_ind, seq[trbc_overlap_start_ind:trbc_overlap_end_ind], cdr3b_rev_primer, cdr3b_rev_out_primer, pm.calc_tm(temp_cdr)

beta = beta.copy()
beta[['TRBC_overlap_start_ind', 'TRBC_overlap_end_ind', 'TRBC_overlap_seq', 'CDR3B_Reverse_Primer', 'CDR3B_Outer_Reverse_Primer', 'CDR3B_rev_overlap_Tm']] = beta.apply(
    lambda row: pd.Series(CDRB_reverse(row['Seq'], row['Prim_overlap_start_ind'], row['Prim_overlap_end_ind'], row['CC'])),
    axis=1
)
#beta = beta.dropna()
beta  
    

,CC,Seq,TRBV_overlap_start_ind,TRBV_overlap_end_ind,TRBV_overlap_seq,Prim_overlap_start_ind,Prim_overlap_end_ind,Primer_overlap_seq,Primer_overlap_Tm,CDR3b_forward,TRBC_overlap_start_ind,TRBC_overlap_end_ind,TRBC_overlap_seq,CDR3B_Reverse_Primer,CDR3B_Outer_Reverse_Primer,CDR3B_rev_overlap_Tm
0,>clonotype1_consensus_1,GGGAGACCTTGCCTATGGGACCATAGGATGCTAGCATACCCCTCCT...,436.0,452.0,TGCCAGCAGCTTAGCC,457.0,476.0,GGACTCCGACTACACCTTC,54.489626,TGCCAGCAGCTTAGCCGGGGGGGACTCCGACTACACCTTC,504.0,525.0,AGGATCTGAGAAATGTGACTC,TTTCTCAGATCCTCTATTACCAAAAGCCTGGTCCCTGAGCCGAAGG...,GAGTCACATTTCTCAGATCCTCTATTACCAAAAG,54.568380
2,>clonotype2_consensus_1,TCTCGTGCTTACGTGGAGTTTCTATGAGTGAAGCCACTGCCTCATC...,374.0,394.0,GTATCTTTGTGCAAGCAGCT,396.0,414.0,GGCGGATGGAACACCTTG,56.326643,GTATCTTTGTGCAAGCAGCTCTGGCGGATGGAACACCTTG,448.0,469.0,AGGATCTGAGAAATGTGACTC,CAGATCCTCTAGCACCGATAGTCGGGTGCCCGCACCAAAGTACAAG...,GAGTCACATTTCTCAGATCCTCTAGCACCGATAG,54.765029
5,>clonotype3_consensus_1,AATACCCGTCTGGAGCCTGATTCCACCATGAGCTGCAGGCTTCTCC...,352.0,368.0,GTGCCAGCAGCCAAGA,366.0,390.0,GATAACAATTCTGGAAATACGCTC,54.076533,GTGCCAGCAGCCAAGATAACAATTCTGGAAATACGCTC,424.0,445.0,AGGATCTGAGAAATGTGACTC,CTCTACAACAATGAGCCGGCTTCCTTCTCCAAAATAGAGCGTATTT...,GAGTCACATTTCTCAGATCCTCTACAACAATGAGCCGGC,56.885491
7,>clonotype4_consensus_1,GAAAACTGGCTTGCTCATGTAGAGAAGTGGTGGAGTGTCTTAACTG...,476.0,494.0,TTCTGTGCCAGCAGTGAT,491.0,516.0,GATAATAGGGGAGAAGTCTTCTTTG,54.571719,TTCTGTGCCAGCAGTGATAATAGGGGAGAAGTCTTCTTTG,543.0,564.0,AGGATCTGAGAAATGTGACTC,CAGATCCTCTACAACTGTGAGTCTGGTTCCTTTACCAAAGAAGACT...,GAGTCACATTTCTCAGATCCTCTACAACTGTGAGT,54.796917
9,>clonotype5_consensus_1,AATACCCGTCTGGAGCCTGATTCCACCATGAGCTGCAGGCTTCTCC...,350.0,366.0,TTGTGCCAGCAGCCAA,369.0,387.0,GGGCAAGACACCCAGTAC,55.201463,TTGTGCCAGCAGCCAACTGGGGCAAGACACCCAGTAC,418.0,439.0,AGGATCTGAGAAATGTGACTC,TCTCAGATCCTCTAACACGAGGAGCCGAGTGCCTGGCCCAAAGTAC...,GAGTCACATTTCTCAGATCCTCTAACACGAGG,56.330476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21119,>clonotype9960_consensus_1,TGGGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATG...,400.0,416.0,TGTGCCAGCGGTGATG,419.0,439.0,GCTGGGGGTTTAACCAAGAC,56.641294,TGTGCCAGCGGTGATGCGGGCTGGGGGTTTAACCAAGAC,479.0,500.0,AGGATCTGAGAAATGTGACTC,CTAACACGAGGAGCCGAGTGCCTGGCCCAAAGTACTGGGTGTCTTG...,GAGTCACATTTCTCAGATCCTCTAACACGAGGAGCCGAG,54.812132
21121,>clonotype9961_consensus_1,GGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATGTC...,395.0,411.0,TTCTGTGCCAGCGGTG,NaN,NaN,None,NaN,None,NaN,NaN,None,None,None,NaN
21124,>clonotype9963_consensus_1,GAAGCAGGCTGAGCCTAAAGCACAGAAGGGCATAGCCAACACTGTA...,543.0,562.0,TGTACTTCTGTGCCAGCTC,NaN,NaN,None,NaN,None,NaN,NaN,None,None,None,NaN
21126,>clonotype9964_consensus_1,TGGGGCTCTCTCACCAAAGAGACCAGTATCCTGAGAGGAAGCATGT...,399.0,415.0,TGTGCCAGCGGTGATG,419.0,436.0,GGGACTGGGGGGTCATC,56.161153,TGTGCCAGCGGTGATGCAATGGGACTGGGGGGTCATC,481.0,502.0,AGGATCTGAGAAATGTGACTC,AGCACCGATAGTCGGGTGCCCGCACCAAAGTACAAGGTGTTTTGAT...,GAGTCACATTTCTCAGATCCTCTAGCACCGATAGTCGGGT,55.047754


In [14]:
alpha.to_csv('Clonotypes_alpha.csv')

In [15]:
beta.to_csv('Clonotypes_beta.csv')

In [21]:
len(alpha_orig)

10522

In [22]:
len(beta_orig)

10612

In [23]:
len(alpha.dropna())

10049

In [24]:
len(beta.dropna())

9429